# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [93]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [94]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df = cities_df.drop(columns = "Unnamed: 0")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Yellowknife,62.4560,-114.3525,12.72,57,20,13.80,CA,1649731284
1,Qaqortoq,60.7167,-46.0333,31.17,53,76,12.71,GL,1649731437
2,Guerrero Negro,27.9769,-114.0611,60.44,79,0,13.11,MX,1649731438
3,Saldanha,-33.0117,17.9442,62.65,55,0,5.97,ZA,1649731438
4,George Town,5.4112,100.3354,87.73,74,20,5.01,MY,1649731439


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [95]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [96]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map
locations = cities_df[["Lat", "Lng"]]

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df["Humidity"], 
                                 dissipating=False, max_intensity=cities_df["Humidity"].max(),
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [97]:
ideal_cities_df = cities_df.loc[(cities_df["Max Temp"] < 75) & (cities_df["Max Temp"] > 60) & 
                                (cities_df["Wind Speed"] < 5) & (cities_df["Cloudiness"] < 5)]
ideal_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
87,Bredasdorp,-34.5322,20.0403,63.86,72,1,2.53,ZA,1649731474
157,Arandis,-22.4167,14.9667,70.38,35,0,3.04,NaN,1649731505
248,Lüderitz,-26.6481,15.1594,66.97,54,0,4.90,NaN,1649731543
253,Faya,18.3851,42.4509,60.91,50,0,3.69,SA,1649731545
259,Alta Gracia,-31.6529,-64.4283,61.38,67,0,3.78,AR,1649731547


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [108]:
# Create dataframe and empty column
hotel_df = pd.DataFrame(index = ideal_cities_df.index)
hotel_df["Lat"] = ideal_cities_df["Lat"]
hotel_df["Lng"] = ideal_cities_df["Lng"]
hotel_df["Hotel Name"] = ""
hotel_df["City"] = ideal_cities_df["City"]
hotel_df["Country"] = ideal_cities_df["Country"]
hotel_df.head()

,Lat,Lng,Hotel Name,City,Country
87,-34.5322,20.0403,,Bredasdorp,ZA
157,-22.4167,14.9667,,Arandis,NaN
248,-26.6481,15.1594,,Lüderitz,NaN
253,18.3851,42.4509,,Faya,SA
259,-31.6529,-64.4283,,Alta Gracia,AR


In [109]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}

In [110]:
#Get hotel name
for index, row in hotel_df.iterrows():
    location = str(hotel_df.loc[index, "Lat"]) + "," + str(hotel_df.loc[index, "Lng"])
    params['location'] = location
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        print("Couldn't find hotel near this city")
hotel_df.head()

,Lat,Lng,Hotel Name,City,Country
87,-34.5322,20.0403,Firlane House,Bredasdorp,ZA
157,-22.4167,14.9667,Vvt-i Park & Hotel,Arandis,NaN
248,-26.6481,15.1594,Alte Villa,Lüderitz,NaN
253,18.3851,42.4509,Beautat Hotel,Faya,SA
259,-31.6529,-64.4283,Hotel Ritz,Alta Gracia,AR


In [111]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
Name
{Hotel Name}
City
{City}
Country
{Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [112]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, hover_text = hotel_info)

fig.add_layer(marker)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))